In [2]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [27]:
df = pd.read_csv('phq_all_final.csv')
df.drop_duplicates(inplace=True)
df.sort_values(by=['patient_date_created', 'date'], inplace=True)
df.drop(labels=['type'], axis='columns', inplace=True)
df['date'] = pd.to_datetime(df['date'])
df['patient_date_created'] = pd.to_datetime(df['patient_date_created'])
df.head()

,date,patient_id,patient_date_created,score
0,2019-08-26 13:32:43.019162,8430,2019-06-03 13:23:02.635680,6
1,2019-11-15 17:55:11.364877,8430,2019-06-03 13:23:02.635680,1
2,2019-10-07 18:23:29.611351,14338,2019-06-03 13:32:30.668460,10
3,2019-11-04 20:16:28.610965,14338,2019-06-03 13:32:30.668460,7
4,2019-12-02 20:56:55.966354,14338,2019-06-03 13:32:30.668460,9


In [30]:
grouped = df.groupby(['patient_id', 'patient_date_created'], sort=False)
grouped.agg(
    first_date = ('date', 'first'),
    last_date = ('date', 'last'),
    first_score = ('score', 'first'),
    last_score = ('score', 'last'),
    max_score = ('score', 'max'),
    min_score = ('score', 'min'),
    test_count = ('score', 'count')
).reset_index()


,patient_id,patient_date_created,first_date,last_date,first_score,last_score,max_score,min_score,test_count
0,8430,2019-06-03 13:23:02.635680,2019-08-26 13:32:43.019162,2019-11-15 17:55:11.364877,6,1,6,1,2
1,14338,2019-06-03 13:32:30.668460,2019-10-07 18:23:29.611351,2019-12-02 20:56:55.966354,10,9,10,7,3
2,17028,2019-06-03 14:24:06.741810,2019-09-17 14:08:49.642430,2019-11-11 14:42:40.889057,6,9,9,6,2
3,3788,2019-06-03 14:48:09.129756,2019-06-07 13:05:01.435941,2019-08-10 04:44:56.461089,0,0,2,0,7
4,6292,2019-06-03 15:13:41.387077,2019-07-16 14:32:59.141021,2019-09-10 14:19:42.829505,5,1,5,1,3
...,...,...,...,...,...,...,...,...,...
15497,10683,2020-07-31 13:54:20.164253,2020-07-31 13:59:13.201741,2020-07-31 13:59:13.201741,1,1,1,1,1
15498,12344,2020-07-31 15:06:51.043730,2020-07-31 15:08:56.079027,2020-07-31 15:08:56.079027,1,1,1,1,1
15499,17475,2020-07-31 15:40:07.726529,2020-07-31 15:52:50.353265,2020-07-31 15:52:50.353265,7,7,7,7,1
15500,9069,2020-07-31 17:33:27.096572,2020-07-31 17:36:57.057913,2020-07-31 17:36:57.057913,3,3,3,3,1
